# Phase 2: Data Collection

## Jason Zheng, jz449

In [125]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import io
import sys
import datetime
from pandas.core.common import SettingWithCopyWarning
import warnings

In [126]:
response = requests.get("https://api.covid19api.com/all")
data = response.json()
df_uscovid = pd.DataFrame.from_records(data)

In [127]:
df_uscovid = df_uscovid.drop(columns=['CountryCode', 'City', 'CityCode', 'Lat', 'Lon', 'Active'])
df_uscovid = df_uscovid[df_uscovid['Country']=='United States of America']

# Converting date format: https://stackoverflow.com/questions/26763344/convert-pandas-column-to-datetime
df_uscovid['Date'] = pd.to_datetime(df_uscovid['Date'], format="%Y-%m-%dT%H:%M:%SZ")

Looking at the filtered dataframe more closely, we see that there are entries with missing Province values. These entries correspond to U.S totals, which are inherently different than the day-by-day state data. Therefore, we will extract these entries as these are what we want to analyze.

In [128]:
df_uscovid = df_uscovid[df_uscovid['Province']=='']
df_uscovid = df_uscovid.drop(columns=['Province'])

Now we will create columns for daily counts of confirmed, death, and recovered cases.

In [129]:
df_uscovid["Daily Confirmed"] = df_uscovid["Confirmed"].diff()
df_uscovid["Daily Deaths"] = df_uscovid["Deaths"].diff()
df_uscovid["Daily Recovered"] = df_uscovid["Recovered"].diff()

In [130]:
df_uscovid.head()

,Country,Confirmed,Deaths,Recovered,Date,Daily Confirmed,Daily Deaths,Daily Recovered
32102,United States of America,1,0,0,2020-01-22,NaN,NaN,NaN
34191,United States of America,1,0,0,2020-01-23,0.0,0.0,0.0
37030,United States of America,2,0,0,2020-01-24,1.0,0.0,0.0
42691,United States of America,2,0,0,2020-01-25,0.0,0.0,0.0
43003,United States of America,5,0,0,2020-01-26,3.0,0.0,0.0


## Domain Data

### Analysis-Ready Dataset #1

In [131]:
df_domains = pd.read_csv('domains.csv', engine='python')

In [132]:
df_domains = df_domains.drop(columns=['Domain', 'Registration', 'Expires', 'Registrant', 'Administrative Contact', 'Nameservers', 'Registrar Contact', 'WHOIS Server', 'Contact Email'])

In [133]:
df_domains.head()

,Updated,registrarName,Ext.
0,NaN,"GoDaddy.com, LLC",.info
1,2020-03-21T04:58:10Z,"GoDaddy.com, LLC",.com
2,2020-03-20T16:29:34Z,"GoDaddy.com, LLC",.com
3,NaN,NaN,.site
4,2020-02-25T21:16:24Z,"GoDaddy.com, LLC",.com


As we can see, some of the entries in the data have NaN values. Therefore, we will create a new dataframe that drops all NaN entries in the 'Updated' column, as they do not tell us when the website was registered.

In [134]:
df_domains_updated = df_domains.dropna(subset=['Updated'])

In [146]:
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
df_domains_updated['Updated'] = pd.to_datetime(df_domains_updated['Updated'], utc=True, errors='coerce').dt.date

In [136]:
# https://stackoverflow.com/questions/45619371/how-can-i-use-pandas-to-count-values-for-each-date-in-a-dataframe
df_domains_counts = df_domains_updated.groupby('Updated').size()

In [137]:
df_domains_counts = df_domains_counts.to_frame()

In [138]:
df_domains_counts = df_domains_counts.reset_index()

In [139]:
df_domains_counts.columns = ['Date','Count']

In [140]:
# Cumulative sum column: https://stackoverflow.com/questions/41859311/cumsum-as-a-new-column-in-an-existing-pandas-data/41859343
df_domains_counts['Cumulative Count'] = df_domains_counts['Count'].cumsum()

In [141]:
df_domains_counts.tail()

,Date,Count,Cumulative Count
123,2020-04-18,69,48874
124,2020-04-19,88,48962
125,2020-04-20,79,49041
126,2020-04-21,44,49085
127,2020-04-22,14,49099


### Analysis-Ready Dataset #2

To create our second analysis-ready dataset from the domain data (that only contains registrar and extension information), we will simply drop one more column, 'Updated', and drop rows with NaN values in the remaining columns 'registrarName' and 'Ext.'. 

In [142]:
df_domains_regext = df_domains.drop(columns=['Updated'])

In [143]:
df_domains_regext = df_domains_regext.dropna(subset=['registrarName', 'Ext.'])

In [144]:
df_domains_regext.tail()

,registrarName,Ext.
92949,Cronon AG,.info
92952,Mesh Digital Limited,.org
92953,Mesh Digital Limited,.org
92955,home.pl S.A.,.pl
92956,Metaregistrar B.V.,.nl


## Exporting Analysis-Ready Data

In [145]:
df_uscovid.to_json(r'us_covid.json')
df_domains_counts.to_json(r'domaincounts.json')
df_domains_regext.to_json(r'domains.json')